In [ ]:
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover
from autosted.callback_buildingblocks.static_settings import FOVSettingsGenerator
from autosted.callback_buildingblocks.coordinate_value_wrappers import StageOffsetsSettingsGenerator
from autosted.detection import SimpleFocusPlaneDetector
from autosted.callback_buildingblocks.value_wrappers import SimpleManualOffset
from autosted.callback_buildingblocks.data_selection import NewestDataSelector
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion, TimedStoppingCriterion
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.pipeline import AcquisitionPipeline
from autosted.imspector import get_current_stage_coords

In [ ]:
# where to save & whether to save combined HDF5 file
save_folder = 'D:/AUTOMATION/Gabi/GS487_Dppa3_all_mESC/raw'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'C:/Users/RESOLFT/Desktop/config_json/gabi/20240307_590_480_overview.json'

# yx move size between images in spiral
move_size = [50e-6, 50e-6]

# channel to focus in
focus_channel = 0

# manual offset (zyx) to focus
manual_focus_offset = [0, 0, 0]

In [ ]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

In [ ]:
# build pipeline object (1 level: 'field')
pipeline = AcquisitionPipeline(save_folder, ['field'], save_combined_hdf5=save_hdf5, name='multipoint-acquisition')

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('field',
    # 1. load basic measurement parameters from file
    LocationRemover(JSONSettingsLoader(measurement_parameters, as_measurements=False)),
    # 2. (optional) update FOV to match spiral move size in yx (leave z & pixel size as-is -> None)
    FOVSettingsGenerator(lengths=[None] + move_size, pixel_sizes=None),
    # 3. get next position, wrap values as stage offset in parameter dict
    # NOTE: old version, by now, the inner SpiralOffsetGenerator will return a parameter dict by default
    # so this will do the same: SpiralOffsetGenerator(move_size, start_coords[1:])
    StageOffsetsSettingsGenerator(SpiralOffsetGenerator(move_size, start_coords[1:], return_parameter_dict=False)),
    # 4. (optional) adjust focus based on last image
    # nested callbacks are best read inside-out:
    # get latest data, get focus plane, optionally add manual offset, wrap values in offset settings dict
    StageOffsetsSettingsGenerator(
        SimpleManualOffset(
            SimpleFocusPlaneDetector(
                NewestDataSelector(pipeline, level='field'),
                channel=focus_channel
        ), offset=manual_focus_offset))
    )

# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'field')

# set maximum number of acquisitions before stop
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(500))

# start with initial task from callback
pipeline.run(next_position_generator)